In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<h2>Load Paris Data</h3>

In [2]:
req = requests.get('https://fr.geneawiki.com/index.php/Liste_des_quartiers_de_Paris')
soup = BeautifulSoup(req.text, "lxml")

In [3]:
table= soup.find_all('table')[1]

In [4]:
postal_codes = []
boroughs = []
neighborhoods = []

In [5]:
for row in table.find_all("tr")[1:]:  # skipping header row
    cells = row.find_all("td")
    #print(cells)
    postal_code = cells[1].text.strip()
    borough = cells[3].text.rstrip('\n')
    #print(cells[1].text, cells[3].text)
    neighbs = cells[4].dl.find_all('dd')
    for i in range(len(neighbs)):
        #print(neighbs[i].text)
        postal_codes.append(postal_code)
        boroughs.append(borough)
        neighb = neighbs[i].text.strip().split(' - ')[1]
        #some neighbourhoods contain a digit at the end
        #corresponding to the footnote number, I will strip it
        if (neighb[-1].isdigit()):
            neighb = neighb[:-2] #delete space preceeding footnote number too
        neighborhoods.append(neighb.rstrip())

In [6]:
print(postal_codes)

['75001', '75001', '75001', '75001', '75002', '75002', '75002', '75002', '75003', '75003', '75003', '75003', '75004', '75004', '75004', '75004', '75005', '75005', '75005', '75005', '75006', '75006', '75006', '75006', '75007', '75007', '75007', '75007', '75008', '75008', '75008', '75008', '75009', '75009', '75009', '75009', '75010', '75010', '75010', '75010', '75011', '75011', '75011', '75011', '75012', '75012', '75012', '75012', '75013', '75013', '75013', '75013', '75014', '75014', '75014', '75014', '75015', '75015', '75015', '75015', '75016', '75016', '75016', '75016', '75017', '75017', '75017', '75017', '75018', '75018', '75018', '75018', '75019', '75019', '75019', '75019', '75020', '75020', '75020', '75020']


In [8]:
print(boroughs)

[' Le Louvre', ' Le Louvre', ' Le Louvre', ' Le Louvre', ' La Bourse', ' La Bourse', ' La Bourse', ' La Bourse', ' Le Temple', ' Le Temple', ' Le Temple', ' Le Temple', " L'Hôtel-de-Ville", " L'Hôtel-de-Ville", " L'Hôtel-de-Ville", " L'Hôtel-de-Ville", ' Le Panthéon', ' Le Panthéon', ' Le Panthéon', ' Le Panthéon', ' Le Luxembourg', ' Le Luxembourg', ' Le Luxembourg', ' Le Luxembourg', ' Le Palais-Bourbon', ' Le Palais-Bourbon', ' Le Palais-Bourbon', ' Le Palais-Bourbon', " L'Élysée", " L'Élysée", " L'Élysée", " L'Élysée", " L'Opéra", " L'Opéra", " L'Opéra", " L'Opéra", " L'Enclos-Saint-Laurent", " L'Enclos-Saint-Laurent", " L'Enclos-Saint-Laurent", " L'Enclos-Saint-Laurent", ' Popincourt', ' Popincourt', ' Popincourt', ' Popincourt', ' Reuilly', ' Reuilly', ' Reuilly', ' Reuilly', ' Les Gobelins', ' Les Gobelins', ' Les Gobelins', ' Les Gobelins', " L'Observatoire", " L'Observatoire", " L'Observatoire", " L'Observatoire", ' Vaugirard', ' Vaugirard', ' Vaugirard', ' Vaugirard', ' Passy

In [9]:
print(neighborhoods)

["Saint-Germain-l'Auxerrois", 'Les Halles', 'Le Palais-Royal', 'La Place-Vendôme', 'Gaillon', 'Vivienne', 'Le Mail', 'Bonne-Nouvelle', 'Les Arts-et-Métiers', 'Les Enfants-Rouges', 'Les Archives', 'Saint-Avoye', 'Saint-Merri', 'Saint-Gervais', "L'Arsenal", 'Notre-Dame', 'Saint-Victor', 'Le Jardin-des-Plantes', 'Le Val-de-Grâce', 'La Sorbonne', 'La Monnaie', "L'Odéon", 'Notre-Dame-des-Champs', 'Saint-Germain-des-Prés', "Saint-Thomas-d'Aquin", 'Les Invalides', "L'École-Militaire", 'Le Gros-Caillou', 'Les Champs-Élysées', 'Le Faubourg-du-Roule', 'La Madeleine', "L'Europe", 'Saint-Georges', "La Chaussée-d'Antin", 'Le Faubourg-Montmartre', 'Rochechouart', 'Saint-Vincent-de-Paul', 'La Porte-Saint-Denis', 'La Porte-Saint-Martin', "L'Hôpital-Saint-Louis", 'La Folie-Méricourt', 'Saint-Ambroise', 'La Roquette', 'Sainte-Marguerite', 'Le Bel-Air', 'Picpus', 'Bercy', 'Les Quinze-Vingts', 'La Salpêtrière', 'La Gare', 'La Maison-Blanche', 'Croulebarbe', 'Le Montparnasse', 'Parc Montsouris', 'Le Petit-

In [12]:
paris_df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
paris_df.PostalCode = postal_codes
paris_df.Borough = boroughs
paris_df.Neighborhood = neighborhoods
print(paris_df.shape)
paris_df.head(100)

(80, 3)


,PostalCode,Borough,Neighborhood
0,75001,Le Louvre,Saint-Germain-l'Auxerrois
1,75001,Le Louvre,Les Halles
2,75001,Le Louvre,Le Palais-Royal
3,75001,Le Louvre,La Place-Vendôme
4,75002,La Bourse,Gaillon
5,75002,La Bourse,Vivienne
6,75002,La Bourse,Le Mail
7,75002,La Bourse,Bonne-Nouvelle
8,75003,Le Temple,Les Arts-et-Métiers
9,75003,Le Temple,Les Enfants-Rouges


In [28]:
#paris_df.to_csv('paris_data_acquisition.csv', index= False)
#paris_df = pd.read_csv('paris_data_acquisition.csv')

In [13]:
address = "75000, Paris, France"

geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [14]:
cols = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
paris_coords_df = pd.DataFrame(columns = cols)
paris_coords_df['PostalCode'] = paris_df['PostalCode']
paris_coords_df['Borough'] = paris_df['Borough']
paris_coords_df['Neighborhood'] = paris_df['Neighborhood']
latitudes = []
longitudes = []
for index, row in paris_df.iterrows():
    address = "{}, Paris, {}".format(row["PostalCode"], row["Neighborhood"])
    #print ("processing: ", address)
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
paris_coords_df['Latitude'] = latitudes
paris_coords_df['Longitude'] = longitudes

In [15]:
paris_coords_df.head(80)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,75001,Le Louvre,Saint-Germain-l'Auxerrois,48.859696,2.340633
1,75001,Le Louvre,Les Halles,48.862157,2.344796
2,75001,Le Louvre,Le Palais-Royal,48.863585,2.336204
3,75001,Le Louvre,La Place-Vendôme,48.867322,2.329483
4,75002,La Bourse,Gaillon,48.869135,2.332909
5,75002,La Bourse,Vivienne,48.868859,2.339363
6,75002,La Bourse,Le Mail,48.868054,2.344595
7,75002,La Bourse,Bonne-Nouvelle,48.870623,2.348750
8,75003,Le Temple,Les Arts-et-Métiers,48.865462,2.355330
9,75003,Le Temple,Les Enfants-Rouges,48.863030,2.361253


In [16]:
paris_coords_df.to_csv('./Data/paris_coords_acquisition.csv', index= False)
#paris_coords_df = pd.read_csv('paris_coords_acquisition.csv')

<h2>Acquire coordinates for greater Paris (suburbs - Ile-de-France region)</h2>

In [17]:
req2 = requests.get('https://www.annuaire-administration.com/code-postal/region/ile-de-france.html')
soup2 = BeautifulSoup(req2.text, "lxml")

In [18]:
table2= soup2.find_all('table')[1]

Paris greater area consists of 7 districts (départements) which I will consider as boroughs for the purposes of this analysis. Small towns and villages will be considered as neighborhoods.

In [19]:
greater_paris_postal_codes = []
greater_paris_boroughs = []
greater_paris_neighborhoods = []

In [20]:
greater_paris_boroughs_dict = {"77": "Seine-et-Marne", "78": "Yvelines", "91": "Essonne", "92": "Hauts-de-Seine",\
                                "93": "Seine-Saint-Denis","94": "Val-de-Marne", "95": "Val-d'Oise"}

In [36]:
import re

#skip 28 rows corresponding to header and Paris boroughs which we already have
for row in table2.find_all("tr")[28:]:
    cells = row.find_all("td")
    if(len(cells)> 0):
        #take only first occurence of boroughsboroughs are mentioned for the second
        #time in cells containing several postal codes
        #I will filter out those cells checking the length of the result found
        #if this length is 1, then it is the first occurence which I take
        p_codes = re.findall(r'\d+', cells[0].text)
        #Boroughs inside Paris postal codes start from 75000
        #suburbs postal codes start from 77000
        #here I filter out only suburbs (greater Paris)
        if (len(p_codes) == 1 and int(p_codes[0]) >= 77000):
            p_code = p_codes[0]
            neighbs = cells[1].text.strip().split('\n')
            for i in range(len(neighbs)):
                greater_paris_postal_codes.append(p_code)
                #two first digits of the code give the borough (département) number
                #we get its name from the boroughs dictionary
                greater_paris_boroughs.append(greater_paris_boroughs_dict[p_code[:2]])
                greater_paris_neighborhoods.append(neighbs[i])
                

In [37]:
greater_paris_df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
greater_paris_df.PostalCode = greater_paris_postal_codes
greater_paris_df.Borough = greater_paris_boroughs
greater_paris_df.Neighborhood = greater_paris_neighborhoods
print(greater_paris_df.shape)
greater_paris_df.head()

(1280, 3)


,PostalCode,Borough,Neighborhood
0,77000,Seine-et-Marne,Livry-sur-Seine
1,77000,Seine-et-Marne,Melun
2,77000,Seine-et-Marne,La Rochette
3,77000,Seine-et-Marne,Vaux-le-Pénil
4,77090,Seine-et-Marne,Collégien


<h3>Acquire coordinates for Greater Paris</h3>

Trying to get all the coordinates in one go generates a "Two many requests error". For this reason, I will get the coordinates by boroughs.

In [38]:
def get_coords(codes):
    """
    arguments: boroughs: list
    returns dataframe with coordinates
    """
    cols = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
    greater_paris_coords_df = pd.DataFrame(columns = cols)
    postal_codes = []
    boroughs = []
    neighborhoods = []
    latitudes = []
    longitudes = []
    for index, row in greater_paris_df.iterrows():
        if (row["PostalCode"][:2] in codes):
            address = "{}, {}, {}, France".format(row["PostalCode"], row["Borough"], row["Neighborhood"])
            print ("processing: ", address)
            location = geolocator.geocode(address)
            postal_codes.append(row['PostalCode'])
            boroughs.append(row['Borough'])
            neighborhoods.append(row['Neighborhood'])
            if (location is not None):
                latitudes.append(location.latitude)
                longitudes.append(location.longitude)
            else:
                latitudes.append(None)
                longitudes.append(None)
    greater_paris_coords_df['PostalCode'] = postal_codes
    greater_paris_coords_df['Borough'] = boroughs
    greater_paris_coords_df['Neighborhood'] = neighborhoods
    greater_paris_coords_df['Latitude'] = latitudes
    greater_paris_coords_df['Longitude'] = longitudes
    return greater_paris_coords_df

<h3>Get coordinates for boroughs 92, 93 and 94 which are closest to paris and smaller in size</h3>

In [39]:
coords_92_93_94_df = get_coords(['92', '93', '94'])

processing:  92000, Hauts-de-Seine, Nanterre, France
processing:  92100, Hauts-de-Seine, Boulogne-Billancourt, France
processing:  92110, Hauts-de-Seine, Clichy, France
processing:  92120, Hauts-de-Seine, Montrouge, France
processing:  92130, Hauts-de-Seine, Issy-les-Moulineaux, France
processing:  92140, Hauts-de-Seine, Clamart, France
processing:  92150, Hauts-de-Seine, Suresnes, France
processing:  92160, Hauts-de-Seine, Antony, France
processing:  92170, Hauts-de-Seine, Vanves, France
processing:  92190, Hauts-de-Seine, Meudon, France
processing:  92200, Hauts-de-Seine, Neuilly-sur-Seine, France
processing:  92210, Hauts-de-Seine, Saint-Cloud, France
processing:  92220, Hauts-de-Seine, Bagneux, France
processing:  92230, Hauts-de-Seine, Gennevilliers, France
processing:  92240, Hauts-de-Seine, Malakoff, France
processing:  92250, Hauts-de-Seine, La Garenne-Colombes, France
processing:  92260, Hauts-de-Seine, Fontenay-aux-Roses, France
processing:  92270, Hauts-de-Seine, Bois-Colomb

In [40]:
print(coords_92_93_94_df.shape)
coords_92_93_94_df.head()

(123, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,92000,Hauts-de-Seine,Nanterre,48.888914,2.195851
1,92100,Hauts-de-Seine,Boulogne-Billancourt,48.832237,2.251252
2,92110,Hauts-de-Seine,Clichy,48.903077,2.306128
3,92120,Hauts-de-Seine,Montrouge,48.817588,2.320983
4,92130,Hauts-de-Seine,Issy-les-Moulineaux,48.827983,2.264154


In [21]:
#coords_92_93_94_df.to_csv('./Data/coords_acquisition_92_93_94_df.csv', index= False)
coords_92_93_94_df = pd.read_csv('./Data/coords_acquisition_92_93_94_df.csv')

<h3>Coordinates acquision for borough 77</h3>

In [42]:
coords_77_df = get_coords(['77'])

processing:  77000, Seine-et-Marne, Livry-sur-Seine, France
processing:  77000, Seine-et-Marne, Melun, France
processing:  77000, Seine-et-Marne, La Rochette, France
processing:  77000, Seine-et-Marne, Vaux-le-Pénil, France
processing:  77090, Seine-et-Marne, Collégien, France
processing:  77100, Seine-et-Marne, Mareuil-lès-Meaux, France
processing:  77100, Seine-et-Marne, Meaux, France
processing:  77100, Seine-et-Marne, Nanteuil-lès-Meaux, France
processing:  77111, Seine-et-Marne, Soignolles-en-Brie, France
processing:  77111, Seine-et-Marne, Solers, France
processing:  77114, Seine-et-Marne, Gouaix, France
processing:  77114, Seine-et-Marne, Hermé, France
processing:  77114, Seine-et-Marne, Noyen-sur-Seine, France
processing:  77114, Seine-et-Marne, Villiers-sur-Seine, France
processing:  77115, Seine-et-Marne, Blandy, France
processing:  77115, Seine-et-Marne, Sivry-Courtry, France
processing:  77118, Seine-et-Marne, Balloy, France
processing:  77118, Seine-et-Marne, Bazoches-lès-

GeocoderTimedOut: Service timed out

In [22]:
#coords_77_df.to_csv('coords_acquisition_77_df.csv', index= False)
coords_77_df = pd.read_csv('./Data/coords_acquisition_77_df.csv')
print(coords_77_df.shape)
coords_77_df.head()

(514, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,77000,Seine-et-Marne,Livry-sur-Seine,NaN,NaN
1,77000,Seine-et-Marne,Melun,48.514271,2.668448
2,77000,Seine-et-Marne,La Rochette,48.514271,2.668448
3,77000,Seine-et-Marne,Vaux-le-Pénil,NaN,NaN
4,77090,Seine-et-Marne,Collégien,48.835046,2.668489


<h3>Coordinates acquisition for borough 78</h3>

In [ ]:
coords_78_df = get_coords(['78'])

In [23]:
#coords_78_df.to_csv('coords_acquisition_78_df.csv', index= False)
coords_78_df = pd.read_csv('./Data/coords_acquisition_78_df.csv')
print(coords_78_df.shape)
coords_78_df.head()

(262, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,78000,Yvelines,Versailles,48.804148,2.141432
1,78100,Yvelines,Saint-Germain-en-Laye,48.894447,2.071765
2,78110,Yvelines,Le Vésinet,48.894424,2.127150
3,78111,Yvelines,Dammartin-en-Serve,48.906133,1.619133
4,78112,Yvelines,Fourqueux,48.890315,2.059763


<h3>Coordinates acquisition for borough 91</h3>

In [ ]:
coords_91_df = get_coords(['91'])

In [24]:
#coords_91_df.to_csv('coords_acquisition_91_df.csv', index= False)
coords_91_df = pd.read_csv('./Data/coords_acquisition_91_df.csv')
print(coords_91_df.shape)
coords_91_df.head()

(196, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,91000,Essonne,Évry,48.624190,2.445880
1,91070,Essonne,Bondoufle,48.611129,2.387138
2,91080,Essonne,Courcouronnes,48.623463,2.414525
3,91090,Essonne,Lisses,48.600048,2.431137
4,91100,Essonne,Corbeil-Essonnes,48.610746,2.476254


<h3>Coordinates acquisition for borough 95</h3>

In [ ]:
coords_95_df = get_coords(['95'])

In [25]:
#coords_95_df.to_csv('coords_acquisition_95_df.csv', index= False)
coords_95_df = pd.read_csv('./Data/coords_acquisition_95_df.csv')
print(coords_95_df.shape)
coords_95_df.head()

(185, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,95000,Val-d'Oise,Boisemont,49.022400,2.002638
1,95000,Val-d'Oise,Cergy,49.027836,2.065427
2,95000,Val-d'Oise,Neuville-sur-Oise,49.013553,2.062368
3,95000,Val-d'Oise,Pontoise,49.027836,2.065427
4,95100,Val-d'Oise,Argenteuil,48.948703,2.239321


<h3>Concatenate all boroughs dataframes</h3>

In [26]:
frames = [paris_coords_df, coords_77_df, coords_78_df, coords_91_df, coords_92_93_94_df, coords_95_df]
greater_paris_coords_df = pd.concat(frames)

In [27]:
print(greater_paris_coords_df.shape)
greater_paris_coords_df.head()

(1360, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,75001,Le Louvre,Saint-Germain-l'Auxerrois,48.859696,2.340633
1,75001,Le Louvre,Les Halles,48.862157,2.344796
2,75001,Le Louvre,Le Palais-Royal,48.863585,2.336204
3,75001,Le Louvre,La Place-Vendôme,48.867322,2.329483
4,75002,La Bourse,Gaillon,48.869135,2.332909


<h3>Check the greater Paris coordinates for null values</h3>

In [28]:
greater_paris_coords_df.isnull().sum()

PostalCode        0
Borough           0
Neighborhood      0
Latitude        237
Longitude       237
dtype: int64

In some cases I was unable to load coordinates for some neighborhoods, I will delete the corresponding rows.

In [29]:
print("shape before deleting null values: ", greater_paris_coords_df.shape)
greater_paris_coords_df_2 = greater_paris_coords_df.dropna()
print("shape after deleting null values: ", greater_paris_coords_df_2.shape)
print("rows deleted: ", greater_paris_coords_df.shape[0] - greater_paris_coords_df_2.shape[0])

shape before deleting null values:  (1360, 5)
shape after deleting null values:  (1123, 5)
rows deleted:  237


In [30]:
greater_paris_coords_df_2.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

I am left with 1123 neighborhoods which is still enough for my analysis

In [ ]:
greater_paris_coords_df_2.to_csv('./Data/greater_paris_coords_all_boroughs_df.csv', index= False)
#greater_paris_coords_df_2 = pd.read_csv('greater_paris_coords_all_boroughs_df.csv')

<h3>Map of Greater Paris</h3>

In [31]:
# create map of Paris using latitude and longitude values
map_greater_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(greater_paris_coords_df_2['Latitude'], greater_paris_coords_df_2['Longitude'], greater_paris_coords_df_2['Borough'], greater_paris_coords_df_2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_greater_paris)  
    
map_greater_paris

In [32]:
map_greater_paris.save('map_greater_paris.html')

<h2>Get venues</h2>

In [1]:
CLIENT_ID = 'Your Client_Id' # your Foursquare ID
CLIENT_SECRET = 'Your Client Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Your Client_Id
CLIENT_SECRET:Your Client Secret


In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #name = name.replace("'","\'")
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if(results is not None):       
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

<h3>I will try to get venues separately by boroughs, since I am getting an error in FourSquare when trying to process all the neighborhoods in one go</h3>

<h3>Venues in Paris, postal codes starting with 75</h3>

In [56]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
venues_75 = getNearbyVenues(names=paris_coords_df['Neighborhood'],
                                   latitudes=paris_coords_df['Latitude'],
                                   longitudes=paris_coords_df['Longitude']
                                  )

Saint-Germain-l'Auxerrois
Les Halles
Le Palais-Royal
La Place-Vendôme
Gaillon
Vivienne
Le Mail
Bonne-Nouvelle
Les Arts-et-Métiers
Les Enfants-Rouges
Les Archives
Saint-Avoye
Saint-Merri
Saint-Gervais
L'Arsenal
Notre-Dame
Saint-Victor
Le Jardin-des-Plantes
Le Val-de-Grâce
La Sorbonne
La Monnaie
L'Odéon
Notre-Dame-des-Champs
Saint-Germain-des-Prés
Saint-Thomas-d'Aquin
Les Invalides
L'École-Militaire
Le Gros-Caillou
Les Champs-Élysées
Le Faubourg-du-Roule
La Madeleine
L'Europe
Saint-Georges
La Chaussée-d'Antin
Le Faubourg-Montmartre
Rochechouart
Saint-Vincent-de-Paul
La Porte-Saint-Denis
La Porte-Saint-Martin
L'Hôpital-Saint-Louis
La Folie-Méricourt
Saint-Ambroise
La Roquette
Sainte-Marguerite
Le Bel-Air
Picpus
Bercy
Les Quinze-Vingts
La Salpêtrière
La Gare
La Maison-Blanche
Croulebarbe
Le Montparnasse
Parc Montsouris
Le Petit-Montrouge
La Plaisance
Saint-Lambert
Necker
Grenelle
Javel
Auteuil
La Muette
La Porte-Dauphine
Chaillot
Les Ternes
La Plaine-Monceau
Batignolles
Les Épinettes
Les G

In [58]:
print(venues_75.shape)
venues_75.head()

(5878, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Saint-Germain-l'Auxerrois,48.859696,2.340633,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
1,Saint-Germain-l'Auxerrois,48.859696,2.340633,Place du Louvre,48.859841,2.340822,Plaza
2,Saint-Germain-l'Auxerrois,48.859696,2.340633,Coffee Crêpes,48.858841,2.340802,Coffee Shop
3,Saint-Germain-l'Auxerrois,48.859696,2.340633,Église Saint-Germain l'Auxerrois,48.859887,2.340617,Church
4,Saint-Germain-l'Auxerrois,48.859696,2.340633,Le Fumoir,48.860341,2.340647,Cocktail Bar


In [ ]:
venues_75.to_csv('./Data/venues_75.csv', index= False)

<h3>Venues in boroughs 92, 93 and 94 which are closest to Paris</h3>

In [ ]:
venues_92_93_94 = getNearbyVenues(names=coords_92_93_94_df_2['Neighborhood'],
                                   latitudes=coords_92_93_94_df_2['Latitude'],
                                   longitudes=coords_92_93_94_df_2['Longitude']
                                  )

In [59]:
#venues_92_93_94.to_csv('venues_92_93_94.csv', index= False)
venues_92_93_94 = pd.read_csv('./Data/venues_92_93_94.csv')

In [60]:
print(venues_92_93_94.shape)
venues_92_93_94.head()

(1174, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Nanterre,48.888914,2.195851,L'Apostrophe,48.891017,2.195062,Bistro
1,Nanterre,48.888914,2.195851,Miki Sushi,48.891543,2.195448,Japanese Restaurant
2,Nanterre,48.888914,2.195851,Romaric,48.890328,2.195096,Bakery
3,Nanterre,48.888914,2.195851,Parc des Anciennes Mairies,48.891485,2.197019,Park
4,Nanterre,48.888914,2.195851,Hokiko,48.885778,2.197173,Japanese Restaurant


<h3>Venues in 77</h3>

In [ ]:
coords_77_df = pd.read_csv('coords_acquisition_77_df.csv')
coords_77_df.head()
print("shape before deleting null values: ", coords_77_df.shape)
coords_77_df_2 = coords_77_df.dropna()
print("shape after deleting null values: ", coords_77_df_2.shape)
print("rows deleted: ", coords_77_df.shape[0] - coords_77_df_2.shape[0])
venues_77 = getNearbyVenues(names=coords_77_df_2['Neighborhood'],
                                   latitudes=coords_77_df_2['Latitude'],
                                   longitudes=coords_77_df_2['Longitude']
                                  )

In [61]:
#venues_77.to_csv('venues_77.csv', index= False)
venues_77 = pd.read_csv('./Data/venues_77.csv')
print(venues_77.shape)
venues_77.head()

(582, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Collégien,48.835046,2.668489,Jablines,48.833715,2.670815,Lake
1,Collégien,48.835046,2.668489,McDonald's,48.836497,2.662425,Fast Food Restaurant
2,Collégien,48.835046,2.668489,Leroy Merlin Collegien,48.834978,2.662511,Furniture / Home Store
3,Collégien,48.835046,2.668489,Okaidi-Obaibi,48.837160,2.671191,Kids Store
4,Meaux,48.951832,2.890500,Parc du Patis,48.948748,2.885682,Scenic Lookout


In [ ]:
coords_78_df = pd.read_csv('coords_acquisition_78_df.csv')
coords_78_df.head()
print("shape before deleting null values: ", coords_78_df.shape)
coords_78_df_2 = coords_78_df.dropna()
print("shape after deleting null values: ", coords_78_df_2.shape)
print("rows deleted: ", coords_78_df.shape[0] - coords_78_df_2.shape[0])
venues_78 = getNearbyVenues(names=coords_78_df_2['Neighborhood'],
                                   latitudes=coords_78_df_2['Latitude'],
                                   longitudes=coords_78_df_2['Longitude']
                                  )

In [62]:
#venues_78.to_csv('venues_78.csv', index= False)
venues_78 = pd.read_csv('./Data/venues_78.csv')
print(venues_78.shape)
venues_78.head()

(555, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Versailles,48.804148,2.141432,Piscine de Montbauron VertMarine,48.802914,2.140736,Pool
1,Versailles,48.804148,2.141432,La Crêperie,48.805655,2.135598,Creperie
2,Versailles,48.804148,2.141432,Hotel Mercure Versailles Chateau,48.803276,2.134792,Hotel
3,Versailles,48.804148,2.141432,Les burgers de papa,48.805779,2.135264,Fast Food Restaurant
4,Versailles,48.804148,2.141432,Nicolas,48.805382,2.142074,Wine Shop


<h3>Venues in borough 91</h3>

In [ ]:
coords_91_df = pd.read_csv('coords_acquisition_91_df.csv')
coords_91_df.head()
print("shape before deleting null values: ", coords_91_df.shape)
coords_91_df_2 = coords_91_df.dropna()
print("shape after deleting null values: ", coords_91_df_2.shape)
print("rows deleted: ", coords_91_df.shape[0] - coords_91_df_2.shape[0])
venues_91 = getNearbyVenues(names=coords_91_df_2['Neighborhood'],
                                   latitudes=coords_91_df_2['Latitude'],
                                   longitudes=coords_91_df_2['Longitude']
                                  )

In [63]:
#venues_91.to_csv('venues_91.csv', index= False)
venues_91 = pd.read_csv('./Data/venues_91.csv')
print(venues_91.shape)
venues_91.head()

(333, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Évry,48.62419,2.44588,AbsINThe,48.624232,2.443806,Bar
1,Évry,48.62419,2.44588,Restaurant Telecom,48.623885,2.446406,Cafeteria
2,Évry,48.62419,2.44588,RER Bras de Fer — Évry Génopole [D],48.623268,2.450697,Train Station
3,Évry,48.62419,2.44588,Hotel Balladins,48.621824,2.448877,Hotel
4,Évry,48.62419,2.44588,Cafe Bar Restaurant Agri,48.622894,2.450194,Restaurant


<h3>Venues in borough 95</h3>

In [ ]:
coords_95_df = pd.read_csv('coords_acquisition_95_df.csv')
coords_95_df.head()
print("shape before deleting null values: ", coords_95_df.shape)
coords_95_df_2 = coords_95_df.dropna()
print("shape after deleting null values: ", coords_95_df_2.shape)
print("rows deleted: ", coords_95_df.shape[0] - coords_95_df_2.shape[0])
venues_95 = getNearbyVenues(names=coords_95_df_2['Neighborhood'],
                                   latitudes=coords_95_df_2['Latitude'],
                                   longitudes=coords_95_df_2['Longitude']
                                  )

In [64]:
#venues_95.to_csv('venues_95.csv', index= False)
venues_95 = pd.read_csv('./Data/venues_95.csv')
print(venues_95.shape)
venues_95.head()

(332, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Boisemont,49.022400,2.002638,Du bon côté de la crêpe,49.025179,1.999458,Creperie
1,Cergy,49.027836,2.065427,The Irish Pub,49.031772,2.063861,Bar
2,Cergy,49.027836,2.065427,Ibis Cergy Le Port,49.031146,2.069505,Hotel
3,Cergy,49.027836,2.065427,Port Cergy,49.030978,2.060638,Port
4,Neuville-sur-Oise,49.013553,2.062368,Café Le Village,49.014690,2.058738,Bar


<h3>Concatenate venues dataframes</h3>

In [66]:
venue_frames = [venues_75, venues_77, venues_78, venues_91, venues_92_93_94, venues_95]
greater_paris_venues_df = pd.concat(venue_frames)

In [67]:
print(greater_paris_venues_df.shape)
greater_paris_venues_df.shape

(8854, 7)


(8854, 7)

<h3>Check for null values</h3>

In [68]:
greater_paris_venues_df.isnull().sum()

Neighborhood              0
Neighborhood Latitude     0
Neighborhood Longitude    0
Venue                     0
Venue Latitude            0
Venue Longitude           0
Venue Category            0
dtype: int64

No null values found

In [ ]:
greater_paris_venues_df.to_csv('./Data/greater_paris_venues_df.csv', index= False)